In [22]:
import api_keys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests

#### Auth

`api_keys.CLIENT_ID`

`api_keys.CLIENT_SECRET`

#### Spotipy

`sp.featured_playlists`

`sp.audio_features`

In [31]:
sp = spotipy.Spotify(
    client_credentials_manager = SpotifyClientCredentials(
        client_id=api_keys.CLIENT_ID,
        client_secret=api_keys.CLIENT_SECRET
    )    
)

In [34]:
sp.featured_playlists()

{'message': "Editor's picks",
 'playlists': {'href': 'https://api.spotify.com/v1/browse/featured-playlists?timestamp=2019-08-05T18%3A43%3A25&offset=0&limit=20',
  'items': [{'collaborative': False,
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX1gRalH1mWrP'},
    'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX1gRalH1mWrP',
    'id': '37i9dQZF1DX1gRalH1mWrP',
    'images': [{'height': None,
      'url': 'https://pl.scdn.co/images/pl/default/2f02947f1a4acbd139ca82b3288c82e29b6b70bd',
      'width': None}],
    'name': 'Summer Hits',
    'owner': {'display_name': 'Spotify',
     'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
     'href': 'https://api.spotify.com/v1/users/spotify',
     'id': 'spotify',
     'type': 'user',
     'uri': 'spotify:user:spotify'},
    'primary_color': None,
    'public': None,
    'snapshot_id': 'MTU2NDc1NDU2NSwwMDAwMDA3MzAwMDAwMTZjNTJhM2Q3ZmQwMDAwMDE2ODk4YzZiMGFl',
    'tracks': {'href': 'https

`sp.featured_playlists(
    locale = '',
    country = '',
    timestamp = '',
    limit = 50,
    offset = 0
)`

In [11]:
help(sp.audio_features)

Help on method audio_features in module spotipy.client:

audio_features(tracks=[]) method of spotipy.client.Spotify instance
    Get audio features for one or multiple tracks based upon their Spotify IDs
    Parameters:
        - tracks - a list of track URIs, URLs or IDs, maximum: 50 ids

